# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
import sqlite3
import pandas as pd

#Connect to the SQLite database
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

#Check if the connection is established
if conn:
    print("Connection established successfully.")

#Query to get the list of tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

#Fetch all table names
tables = cur.fetchall()

#Check if there are any tables and print the table names
if tables:
    print("Tables in the database:")
    for table in tables:
        print(table[0])
else:
    print("No tables found in the database.")

Connection established successfully.
Tables in the database:
productlines
offices
customers
employees
orderdetails
orders
payments
products


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
# joining employees and offices table
q = """
    SELECT employees.firstName,
           employees.lastName
    FROM employees
    JOIN offices 
    ON employees.officeCode = offices.officeCode;"""      

In [5]:
pd.read_sql(q,conn)

,firstName,lastName
0,Diane,Murphy
1,Mary,Patterson
2,Jeff,Firrelli
3,William,Patterson
4,Gerard,Bondur
5,Anthony,Bow
6,Leslie,Jennings
7,Leslie,Thompson
8,Julie,Firrelli
9,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [11]:
# Your code here
q = """
    SELECT employees.officeCode,
           offices.city,
           COUNT (employees.employeeNumber) AS num_of_employees
    FROM employees
    JOIN offices
    ON employees.officeCode = offices.officeCode 
    GROUP BY employees.officeCode,
             offices.city
    HAVING COUNT (employees.employeeNumber) = 0;"""      

In [12]:
pd.read_sql(q,conn)

,officeCode,city,num_of_employees


## Write 3 questions of your own and answer them

In [17]:
# Answers will vary

# Example question: 
# how many customers are there per office
q="""
SELECT offices.officeCode, 
       offices.city, 
       COUNT(customers.customerNumber) AS num_of_customers
FROM offices 
LEFT JOIN employees ON offices.officeCode = employees.officeCode
LEFT JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
GROUP BY offices.officeCode, offices.city;

"""



In [18]:
pd.read_sql(q,conn)

,officeCode,city,num_of_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17
7,27,Boston,0


In [ ]:
"""
Question 1
"""

# Your code here

In [ ]:
"""
Question 2
"""

# Your code here

In [ ]:
"""
Question 3
"""

# Your code here

## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [25]:
# Your code here
q = """
    SELECT employees.employeeNumber,
       employees.firstName,
       employees.lastName,
       products.productName
FROM 
employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber 
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderDetails ON orders.orderNumber = orderDetails.orderNumber
JOIN products ON orderDetails.productCode = products.productCode
ORDER BY employees.employeeNumber, 
         products.productName;"""

     

In [26]:
pd.read_sql(q,conn)

,employeeNumber,firstName,lastName,productName
0,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
1,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
2,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
3,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
4,1165,Leslie,Jennings,18th Century Vintage Horse Carriage
...,...,...,...,...
2991,1702,Martin,Gerard,The Titanic
2992,1702,Martin,Gerard,The Titanic
2993,1702,Martin,Gerard,The Titanic
2994,1702,Martin,Gerard,The USS Constitution Ship


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [31]:
# Your code here
# Your code here
q = """
    SELECT employees.employeeNumber,
       employees.firstName,
       employees.lastName,
       SUM (orderDetails.quantityOrdered) AS total_products_sold
FROM 
employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber 
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderDetails ON orders.orderNumber = orderDetails.orderNumber
GROUP BY employees.employeeNumber,
         employees.firstName,
         employees.lastName 
 ORDER BY employees.lastName ASC;"""
         

     

In [32]:
pd.read_sql(q,conn)

,employeeNumber,firstName,lastName,total_products_sold
0,1337,Loui,Bondur,6186
1,1501,Larry,Bott,8205
2,1401,Pamela,Castillo,9290
3,1188,Julie,Firrelli,4227
4,1611,Andy,Fixter,6246
5,1702,Martin,Gerard,4180
6,1370,Gerard,Hernandez,14231
7,1165,Leslie,Jennings,11854
8,1504,Barry,Jones,7486
9,1612,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [33]:
# Your code here
query="""
SELECT employees.employeeNumber,
       employees.firstName,
       employees.lastName,
       COUNT(DISTINCT products.productCode) AS uniqueProductsSold
FROM employees
JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders ON customers.customerNumber = orders.customerNumber
JOIN orderDetails ON orders.orderNumber = orderDetails.orderNumber
JOIN products ON orderDetails.productCode = products.productCode
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
HAVING COUNT(DISTINCT products.productCode) > 200
ORDER BY employees.lastName, employees.firstName;
"""

In [35]:
pd.read_sql(query,conn)

,employeeNumber,firstName,lastName,uniqueProductsSold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!